## Cruce de trx con contratos

In [1]:

import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

conf = (SparkConf().set("spark.executor.memory", "6g")
                    .set("spark.executor.cores", "3")
                    .set("spark.executor.instances", "2")
                    .set("spark.eventLog.enabled", "true"))
spark = SparkSession.builder.master("spark://spark-master:7077").appName("JoinTRXContratos").config(conf=conf).getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/22 12:02:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import time
print ('Inicio: '+time.strftime("%c"))
inicio = time.perf_counter()

Inicio: Sat Oct 22 12:02:19 2022


In [3]:
#spark job monitoring
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
from pyspark.sql.functions import sum, col, desc

esquema = StructType() \
      .add("ctId",StringType(),True) \
      .add("fecIni",StringType(),True) \
      .add("fecFin",StringType(),True) \
      .add("dato",StringType(),True) \
      .add("saldo",IntegerType(),True)

df = spark.read.option("mergeSchema", True).option("schema", "esquema").option("basePath", "file:///opt/workspace/datos/fakeContracts01.parquet/")\
               .format("parquet").load('./datos/fakeContracts01.parquet')

In [4]:
print ('Final: '+time.strftime("%c"))
final = time.perf_counter()
print(f'Hecho en {round(final - inicio, 4)} segundo(s)')

Final: Sat Oct 22 12:02:22 2022
Hecho en 2.9497 segundo(s)


In [5]:
esqTRX = StructType() \
      .add("trxId",StringType(),True) \
      .add("contrato",StringType(),True) \
      .add("fecha",StringType(),True) \
      .add("importe",IntegerType(),True) \
      .add("trxTy",StringType(),True)

dfTRX = spark.read.csv("./datos/TRX001.csv", sep=";", header = True, schema = esqTRX)

dfTRX.show()
dfTRX.count()

+-------+------------+----------+-------+--------+
|  trxId|    contrato|     fecha|importe|   trxTy|
+-------+------------+----------+-------+--------+
|TR00001|CT0015331087|2001-03-02|  10000|Da igual|
|TR00002|CT0015331087|2011-03-02|  10002|Da igual|
|TR00003|CT0015331087|2021-03-02|  10003|Da igual|
|TR00004|CT0015331087|2021-08-32|  10004|Da igual|
|TR00005|CT0015326690|2001-03-02|  10000|Da igual|
|TR00006|CT0015326690|2011-03-02|  10000|Da igual|
|TR00007|CT0015326690|2015-03-02|  10000|Da igual|
|TR00008|CT0015326690|2016-03-02|  10000|Da igual|
|TR00009|CT0015326690|2017-03-02|  10000|Da igual|
|TR00010|CT0015326690|2018-03-02|  10000|Da igual|
|TR00011|CT0015326690|2021-03-02|  10000|Da igual|
|TR00012|CT0015326690|2021-04-02|  10000|Da igual|
|TR00014|CT0015326690|2021-06-22|  10000|Da igual|
|TR00205|CT0015346044|2001-03-02|  10000|Da igual|
|TR00206|CT0015346044|2011-03-02|  10000|Da igual|
|TR00207|CT0015346044|2015-03-02|  10000|Da igual|
|TR00208|CT0015346044|2016-03-0

706

In [6]:
dfJ = dfTRX.join(df, (dfTRX.contrato == df.ctId) & (dfTRX.fecha > df.fecIni) & (dfTRX.fecha <= df.fecFin), "left").drop(df.ctId)
dfJ.show(1000)
dfJ.count()

+--------+------------+----------+-------+--------------+----------+----------+--------------------+-----+------+
|   trxId|    contrato|     fecha|importe|         trxTy|    fecIni|    fecFin|                dato|saldo|  proc|
+--------+------------+----------+-------+--------------+----------+----------+--------------------+-----+------+
| TR00005|CT0015326690|2001-03-02|  10000|      Da igual|1999-12-28|2020-02-14|Dato del contrato...| 7200|CT0015|
| TR00006|CT0015326690|2011-03-02|  10000|      Da igual|1999-12-28|2020-02-14|Dato del contrato...| 7200|CT0015|
| TR00007|CT0015326690|2015-03-02|  10000|      Da igual|1999-12-28|2020-02-14|Dato del contrato...| 7200|CT0015|
| TR00008|CT0015326690|2016-03-02|  10000|      Da igual|1999-12-28|2020-02-14|Dato del contrato...| 7200|CT0015|
| TR00009|CT0015326690|2017-03-02|  10000|      Da igual|1999-12-28|2020-02-14|Dato del contrato...| 7200|CT0015|
| TR00010|CT0015326690|2018-03-02|  10000|      Da igual|1999-12-28|2020-02-14|Dato del 

706

In [7]:
spark.sparkContext.stop()
print('Sacabao')

Sacabao
